<a href="https://colab.research.google.com/github/wwwh0225/elements-of-statistical-learning/blob/main/Ch4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4.4 Logistic Regression

羅吉斯迴歸(Logistic Regression)是針對 $K$ 個類別的後驗機率做一個線性的的模型，而跟一般線性迴歸不同的地方是在於，羅吉斯迴歸透過對數的轉換將機率的取值保留在$[0,1]$之間。若我們的資料有 $K$ 類別，則羅吉斯回歸透過建立 $K-1$ 個 **log-odds** 來建構我們所要的模型，少的那一個類別就拿來作為對照組之用。
以 $K$ 作為基準，Logistic Regression 會建立這些模型關係：

$$\begin{aligned} 
\ln \frac{P(G=1|X=x)}{P(G=K|X=x)} &= \beta_{10}+\beta_1^Tx\\
\ln \frac{P(G=2|X=x)}{P(G=K|X=x)} &= \beta_{20}+\beta_2^Tx \\
& \vdots \\
\ln \frac{P(G=K-1|X=x)}{P(G=K|X=x)} &= \beta_{(K-1)0}+\beta_{K-1}^Tx
\end{aligned}$$

我們可以對這些式子取指數回來，可得：

$$P(G=l|X=x) =P(G=K|X=x) e^{\beta_{l0}+\beta_l^Tx}, l=1,2,\cdots,N-1$$

又

$$\begin{aligned} 
P(G=K|X=x) &= 1-\sum_{l=1}^{K-1}P(G=l|X=x)\\&=1-P(G=K|X=x)\sum_{l=1}^{K-1}e^{\beta_{l0}+\beta_l^Tx}
\end{aligned} $$

故可求解出：

$$P(G=K|X=x) = \frac{1}{1+\sum_{l=1}^{K-1}e^{\beta_{l0}+\beta_l^Tx} } $$
以及, 

$$P(G=k|X=x) = \frac{e^{\beta_{k0}+\beta_k^Tx} }{1+\sum_{l=1}^{K-1}e^{\beta_{l0}+\beta_l^Tx} }$$

若我們簡化表示，將這些參數定義為：

$$\theta = \{\beta_{10},\beta_1^T,\cdots,\beta_{(K-1)0},\beta_{(K-1)}^T  \}$$
則類別$k$機率可以表示成： $P(G=k|X=x)=p_k(x;\theta)$

### 配適 Logistic Regression 的參數

在共有 $K$ 類的資料中，可以把資料想成是從**多項分配(multinomial)**取出來的。因此 **likelihood** 可以想成是：

$$L = \prod _{i=1}^Np_{g_i}(x_i)$$

(只有其中一項的指數會是 $1$ ，其他都是 $0$ )
將之轉為 **log-likelihood**：

$$l=\sum _{i=1}^N \ln p_{g_i}(x_i)$$

若我們只先看 $K=2$ 的情形，則：(可參考課本4.4.1)

$$l(\beta) = \sum_{i=1}^N\{y_i\beta^Tx_i -\ln(1+e^{\beta^Tx_i})  \}$$

我們求解 log-likelihood 的極值，另一階導數為0：

$$\frac{\partial l(\beta)}{\partial \beta} = \sum_{i=1}^N x_i(y_i-p(x_i;\beta))=0$$

其中 $p(x_i;\beta) = \frac{e^{\beta^Tx}}{1+e^{\beta^Tx}}$ 。

我們將一階條件，擴寫成矩陣形式：

$$\frac{\partial l(\beta)}{\partial \beta} = \mathbf{X}^T(y-p)$$

二階條件的 **Hessian matrix** 如下：

$$\frac{\partial^2 l(\beta)}{\partial \beta\beta^T} = -\mathbf{X}^T\mathbf{WX}$$

- \mathbf{W} 是第 $i$ 個元素是 $p(x_i;\beta^{old})(1-p(x_i;\beta^{old}))$ 的對角矩陣。

由於此問題的一階條件沒有 closed-form，因此我們使用**牛頓法**找根：

$$\begin{aligned} 
\beta^{new} &= \beta^{old} + (\mathbf{W}^T\mathbf{WX})^{-1}\mathbf{X}^T(\mathbf{y}-\mathbf{p})\\
&= (\mathbf{W}^T\mathbf{WX})^{-1}\mathbf{X}^T\mathbf{W}(\mathbf{X}\beta^{old}+\mathbf{W}^{-1}(\mathbf{y}-\mathbf{p}))\\
&=(\mathbf{W}^T\mathbf{WX})^{-1}\mathbf{X}^T\mathbf{W}\mathbf{z}
\end{aligned}$$

觀察上式，我們令 $\mathbf{z}=\mathbf{X}\beta^{old}+\mathbf{W}^{-1}(\mathbf{y}-\mathbf{p})$，則我們每進行一次迭代，都是彷彿在進行一次 *weighted least squared*，故稱之為 **iteratively reweighted least squared(IRLS)**。在迭代的過程中，我們可以選擇從 $\beta=0$ 開始。


$$\beta^{new} \leftarrow \mathop{\arg\min}_\beta (\mathbf{z}-\mathbf{X}\beta)^T\mathbf{W}(\mathbf{z}-\mathbf{X}\beta)$$

### 一個 $K=2$ 的例子，從 $x \in \mathbb{R}$ 到 $x \in \mathbb{R}^p$ (from Ex. 4.5)

![](https://esl.hohoweiya.xyz/img/04/fig4.16.png)

**Consider a two-class logistic regression problem with $x \in \mathbb{R}$. Characterize the maximum-likelihood estimates of the slope and intercept parameter if the sample $x_i$ for the two classes are separated by a point $x \in \mathbb{R}$. Generalize this result to (a) $x \in \mathbb{R}^p$ (see Figure 4.16), and (b) more than two classes.**

在先前的討論中，我們知道當 $K=2$ 時，log-likelihood function 為

$$l(\beta) = \sum_{i=1}^N\{y_i\beta^Tx_i -\ln(1+e^{\beta^Tx_i})  \}$$

而若 $x \in \mathbb{R}$(一維)，則

$$y_i =\left\{\begin{matrix}
0,\ x_i \leq x_0\\ 
1,\ x_i > x_0
\end{matrix}\right.$$

我們考慮最基本的一維羅吉斯回歸模型，的對數概似函數：

$$l(\beta) = \sum_{i=1}^N\{y_i\beta^Tx_i -\ln(1+e^{\beta^Tx_i})  \}$$

故

$$\begin{aligned}
l(\beta) &= \sum_{i=1}^N\{y_i(\beta_0+\beta_1x_i) -\ln(1+e^{(\beta_0+\beta_1x_i)})  \} \\
&=\sum_{y_i=0}[-\ln(1+e^{(\beta_0+\beta_1x_i)})]+\sum_{y_i=1}[(\beta_0+\beta_1x_i)-\ln(1+e^{(\beta_0+\beta_1x_i)})]
 \end{aligned}$$

若我們想要 **maximize log-likelihood**，很明顯地，當 $\beta \rightarrow \infty$  時就會發散，這導致我們無法找到一個 closed-form 的解。

但很直觀地，當$x \in \mathbb{R}$，由於資料只有一維，那我們在 x 軸上找到 $x=x_0$ ，並在此處畫上一條垂直於 x 軸的線，這樣就能夠明確地劃分出兩個分類。

- 當 $x \in \mathbb{R}^p$ 時，該如何處理呢？(p>1)
  
我們透過一樣的邏輯擴展 $l(\beta)$ 用矩陣向量表示：

$$\begin{aligned}
l(\beta) &= \sum_{i=1}^N\{y_i\beta^Tx_i -\ln(1+e^{\beta^Tx_i})  \} \\
&=\sum_{y_i=0}[-\ln(1+e^{\beta^Tx_i})]+\sum_{y_i=1}[\beta^Tx_i-\ln(1+e^{\beta^Tx_i})]\end{aligned}$$

同樣地，我們同樣會發現 $l(\beta)$ 會發散。

- 當 $K>2$ 時，要如何處理呢？

直觀上來說，當 $K>2$ 則需要超過一個超平面來協助我們分類，我們同樣能寫出這種情況的 log-likelihood function：

$$\begin{aligned}
l(\beta) &= \sum_{i=1}^N [\sum_{k=1}^{K-1}\mathbf{1}_{y_i=k} \beta_k^Tx_i-\ln(1+\sum_{l=1}^{K-1}e^{\beta_l^Tx_i}]\\
&=\sum_{k=1}^{K-1} \sum_{g_k}[\beta_k^Tx_i-\ln(1+\sum_{l=1}^{K-1}\beta_l^Tx_i) ]
+\sum_{g_k}[-\ln(1+\sum_{l=1}^{K-1}e^{\beta_l^Tx_i})]
\end{aligned}$$

而這條式子同樣也找不到 \beta 的 closed-form。

### Example: South African Heart Disease


### Logistic v.s. LDA

